In [1]:
from collections import defaultdict
from dwave.system import DWaveSampler, EmbeddingComposite
import numpy as np
import random
import math

# Edge properties of circulant graph generated by Nijenhuis' algorithm
Here $V=\{0,1,...,a_0-1\}$ \
Let $(j,w) \in B$ ( edge $ 0\rightarrow j$ has weight w ) \
Then $i \rightarrow (i+j)\%a_0$ and $(a_0-j+i)\%a_0 \rightarrow i$  where $i \in V$

In [2]:
# Randomizes input for Frobenius problem
def nums(n=5,r=100):
    out = list()
    l = list(range(2,r))
    gcd = l.pop()
    out.append(gcd)
    while True:
        nl = random.sample(l,n)
        if math.gcd(*nl)==1:
            return nl
# Edge to qbit number encoding
def f(i: int ,j: int) -> int:
    global n
    return n*i+j
# Takes edge's index in B and the vertex at which it must end is 'i'
# Returns the vertex from which the corresponding edge goes to i
def jtoi(j: int ,i: int) -> int:
    global B
    global a0
    return (a0-B[j][0]+i)%a0

In [3]:
def SPConstraintGen(B: list,c: list,a0: int,i: int,Q: defaultdict,flag: int = 3) -> None:
    # B is list of (j,w)
    # a0 is the smallest of input
    # i is the vertex in consideration
    # Q is the input to DWAVE
    # flag : 1 means i is source
    #        2 means i is destination
    #        3 means i is other vertex
    n=len(B)
    Y = range(n)
    a = 0
    b = 0
    if flag==3:
        a=c[3]
        b=c[3]
    elif flag==1:
        a=-c[1]
        b=3*c[1]
    elif flag==2:
        a=3*c[2]
        b=-c[2]
    else:
        return
    
    for j in Y:
        
        z1=f(i,j)
        
        q1=jtoi(j,i)
        z2=f(q1,j)
        
        Q[(z1,z1)]+=a
        Q[(z2,z2)]+=b
        for k in range(j+1,n):
            q2 = jtoi(k,i)
            # i->k
            z3=f(i,k)
            # k->i
            z4=f(q2,k)
            # i->j i->k
            Q[(z1,z3)]+=2*c[flag]
            # j->i k->i
            Q[(z2,z4)]+=2*c[flag]
        for k in Y:
            q2=jtoi(k,i)
            # k->i
            z4=f(q2,k)
            # i->j k->i
            Q[(z1,z4)]+=-2*c[flag]
            

In [4]:

def QShortestPath(B: list,a0: int,s: int,d: int):
    global A
    Q = defaultdict(int)
    c1= [sum(i) for i in zip(*B)][1]*a0
    c=[1,c1,c1,c1]
    X=list(range(a0))
    n=len(B)
    for i in X:
        for j in range(n):
            tmp = f(i,j)
            Q[(tmp,tmp)]=B[j][1]*c[0]
    X.pop(s)
    X.pop(d-1)
    for i in X:
        SPConstraintGen(B=B,c=c,a0=a0,i=i,Q=Q)
    SPConstraintGen(B=B,c=c,a0=a0,i=s,Q=Q,flag=1)
    SPConstraintGen(B=B,c=c,a0=a0,i=d,Q=Q,flag=2)
    s = EmbeddingComposite(DWaveSampler())
    ss = s.sample_qubo(Q,num_reads=999)
    
    O=ss.first[0]
    f1=0
    for i in range(len(B)*a0):
        if O[i]==1:
            f1+=B[i%len(B)][1]
    return f1,ss

In [5]:
def QCirculantDiameter(B: list,a0: int):
    max=0
    l = list()
    for i in range(1,a0):
        t,ss=QShortestPath(B=B,a0=a0,s=0,d=i)
        l.append(ss)
        if t>max:
            max=t
    return max,l

In [6]:
#A=[13,17,19,23,31,37,41,43]
#A=nums()
A=[5,7,11,17]
print('A = ',A)
A.sort(reverse=True)
a0=A.pop()
B=dict()
for x in A:
    i = x%a0
    if i != 0:
        B[i]=x
B=list(B.items())
print(B)
n=len(B)
print('a0 = ',a0)
m,l=QCirculantDiameter(B=B,a0=a0)
print('f(A) = ',m-a0)

A =  [5, 7, 11, 17]
[(2, 7), (1, 11)]
a0 =  5
f(A) =  13


A is the input set. The list after A is B, which contains the general form of edges. $a_0$ is the smallest number in the set A. f(A) is the Frobenius number.